In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
%load_ext autoreload
%autoreload 2
import numpy as np
import pandas as pd

In [2]:
class WeightedGraph:
    def __init__(self):
        self.graph = {}

    def node_exists(self, node):
        """Check if a node exists in the graph."""
        return node in self.graph

    def edge_exists(self, node1, node2):
        """Check if an edge exists between node1 and node2."""
        return self.node_exists(node1) and node2 in self.graph[node1]

    def add_node(self, node):
        """Add a node to the graph."""
        if not self.node_exists(node):
            self.graph[node] = {}

    def add_edge(self, node1, node2, weight):
        """Add an edge between node1 and node2 with the given weight."""
        if not self.edge_exists(node1, node2):
            # If the nodes don't exist, add them to the graph
            self.add_node(node1)
            self.add_node(node2)

            # Add node2 as a neighbor of node1 and vice versa
            self.graph[node1][node2] = weight
            # self.graph[node2][node1] = weight  # Assuming it's an undirected graph

    def __str__(self):
        return str(self.graph)


In [3]:
############################################
### PAGER functions for PAGER server API ###
############################################
import requests

class PAGER():
    def __init__(self):
        self.params = {}
        
    ######   
    # run_pager is a function connected to PAGER api to perform hypergeometric test and retrieve enriched PAGs associated to a list of genes
    # The input parameters are:
    # 1.genes: a list of gene symbols.
    # 2.source: a list of sources refered to 'http://discovery.informatics.uab.edu/PAGER/index.php/pages/help'
    # 3.Type: a list of PAG types consisting of 'P', 'A' or 'G'
    # 4.minSize: the allowed minimum size of PAG genes
    # 5.maxSize: the allowed maximum size of PAG genes
    # 6.similarity: the similarity score cutoff [0,1]
    # 7.overlap: the allowed minimum overlap genes
    # 8.nCoCo: the minimum nCoCo score
    # 9.pValue: p-value cutoff
    # 10.FDR: false discovery rate 
    ######
    def run_pager(self, genes,**kwargs):       
        source = kwargs['source'] if 'source' in kwargs.keys() else ['KEGG_2021_HUMAN', 'WikiPathway_2021', 'BioCarta', 'Reactome_2021', 'Spike']
        Type = kwargs['Type'] if 'Type' in kwargs.keys() else 'All'
        minSize = kwargs['minSize'] if 'minSize' in kwargs.keys() else 1
        maxSize = kwargs['maxSize'] if 'maxSize' in kwargs.keys() else 2000
        sim = kwargs['similarity'] if 'similarity' in kwargs.keys() else 0
        overlap = kwargs['overlap'] if 'overlap' in kwargs.keys() else '1'
        organism = kwargs['organism'] if 'organism' in kwargs.keys() else 'All'
        nCoCo = kwargs['nCoCo'] if 'nCoCo' in kwargs.keys() else '0'
        pvalue = kwargs['pvalue'] if 'pvalue' in kwargs.keys() else 0.05
        FDR = kwargs['FDR'] if 'FDR' in kwargs.keys() else 0.05
        # Set up the call parameters as a dict.
        params = {}
        # Work around PAGER API form encode issue.
        params['genes'] = '%20'.join(genes)
        params['source'] = '%20'.join(source)
        params['type'] = Type
        params['ge'] = minSize
        params['le'] = maxSize        
        params['sim'] = str(sim)
        params['olap'] = str(overlap)
        params['organism'] = organism
        params['cohesion'] = str(nCoCo)
        params['pvalue'] = pvalue
        params['FDR'] = FDR
        response = requests.post('http://discovery.informatics.uab.edu/PAGER/index.php/geneset/pagerapi', data=params)
        #print(response.request.body)
        return pd.DataFrame(response.json())
    
    # pathMember is a function connected to PAGER api to retrieve the membership of PAGs using a list of PAG IDs
    def pathMember(self, PAG_IDs):
        # Set up the call parameters as a dict.
        params = {}
        params['pag'] = ','.join(PAG_IDs)
        # Work around PAGER API form encode issue.
        response = requests.post('http://discovery.informatics.uab.edu/PAGER/index.php/geneset/get_members_by_ids/', data=params)
        #print(response.request.body)
        return pd.DataFrame(response.json()['data'])       
    
    # pathInt is a function connected to PAGER api to retrieve the m-type relationships of PAGs using a list of PAG IDs 
    def pathInt(self, PAG_IDs):
        # Set up the call parameters as a dict.
        params = {}
        params['pag'] = ','.join(PAG_IDs)
        # Work around PAGER API form encode issue.
        response = requests.post('http://discovery.informatics.uab.edu/PAGER/index.php/pag_pag/inter_network_int_api/', data=params)
        #print(response.request.body)
        return pd.DataFrame(response.json()['data'])
    
    # pathReg is a function connected to PAGER api to retrieve the r-type relationships of PAGs using a list of PAG IDs   
    def pathReg(self, PAG_IDs):
        # Set up the call parameters as a dict.
        params = {}
        params['pag'] = ','.join(PAG_IDs)
        # Work around PAGER API form encode issue.
        response = requests.post('http://discovery.informatics.uab.edu/PAGER/index.php/pag_pag/inter_network_reg_api/', data=params)
        #print(response.request.body)
        return pd.DataFrame(response.json()['data'])
    
    # pagRankedGene is a function connected to PAGER api to retrieve RP-ranked genes with RP-score of the given PAG_IDs
    def pagRankedGene(self, PAGid):
        response = requests.get('http://discovery.informatics.uab.edu/PAGER/index.php/genesinPAG/viewgenes/'+PAGid)
        return pd.DataFrame(response.json()['gene'])
    
    # pagGeneInt is a function connected to PAGER api to retrieve gene interaction network
    def pagGeneInt(self, PAGid):
        response = requests.get('http://discovery.informatics.uab.edu/PAGER/index.php/pag_mol_mol_map/interactions/'+PAGid)
        return pd.DataFrame(response.json()['data'])
    
    # pagGeneReg is a function connected to PAGER api to retrieve gene regulatory network
    def pagGeneReg(self, PAGid):
        response = requests.get('http://discovery.informatics.uab.edu/PAGER/index.php/pag_mol_mol_map/regulations/'+PAGid)
        return pd.DataFrame(response.json()['data'])
    
    # path_NGSEA is a function connected to PAGER api to generate the network-based GSEA result
    import requests
    def path_NGSEA(SELF, genes, PAGmember):
        geneExpStr = ''
        for rowIdx in range(0,genes.shape[0]):
            geneExpStr = geneExpStr + genes.iloc[rowIdx,0]+'\\t\\t'+genes.iloc[rowIdx,1] + "\\t\\t\\t"
        PAGsetsStr = ''
        for rowIdx in range(0,PAGmember.shape[0]):
            PAGsetsStr = PAGsetsStr + PAGmember.iloc[rowIdx,0]+'\\t\\t'+PAGmember.iloc[rowIdx,1] + "\\t\\t\\t"
        params = {}
        params['geneExpStr'] = geneExpStr
        params['PAGsetsStr'] = PAGsetsStr
        response = requests.post('http://discovery.informatics.uab.edu/PAGER/index.php/geneset/ngseaapi/', data=params)
        #print(response.request.body)
        return pd.DataFrame(response.json()['data'])


In [4]:
PAGER = pager.PAGER()

NameError: name 'pager' is not defined

In [ ]:
######   
# run_pager is a function connected to PAGER api to perform hypergeometric test and retrieve enriched PAGs associated to a list of genes
# The input parameters are:
# 1.genes: a list of gene symbols.
# 2.source: a list of sources refered to 'http://discovery.informatics.uab.edu/PAGER/index.php/pages/help'
# 3.Type: a list of PAG types consisting of 'P', 'A' or 'G'
# 4.minSize: the allowed minimum size of PAG genes
# 5.maxSize: the allowed maximum size of PAG genes
# 6.similarity: the similarity score cutoff [0,1]
# 7.overlap: the allowed minimum overlap genes
# 8.nCoCo: the minimum nCoCo score
# 9.pValue: p-value cutoff
# 10.FDR: false discovery rate 
######
PAG_result = PAGER.run_pager(
    ['BRCA1','BRCA2'],
    source = ['KEGG_2021_HUMAN', 'WikiPathway_2021', 'BioCarta', 'Reactome_2021', 'Spike'],
    Type='All',
    minSize=1,
    maxSize=2000,
    similarity = 0.05,
    overlap = 1,
    nCoCo = 0,
    pValue = 0.05,
    FDR = 0.05
)
PAG_result

,GS_ID,NAME,SOURCE,GS_SIZE,ORGANISM,DESCRIPTION,LINK,TYPE,MULTI_N,OLAP,COCO_V2,SIMILARITY_SCORE,pvalue,Rank,pFDR
0,WAG002693,Sandbox pathway test,WikiPathway_2021,5,Homo sapiens,Sandbox pathway test,https://www.wikipathways.org/index.php/Pathway...,P,1333,2,188.558607956135,.52,1.020749e-08,1,0.000001
1,WAG000644,role of brca1 brca2 and atr in cancer suscepti...,BioCarta,21,Homo sapiens,BRCA1 and BRCA2 were identified genetically as...,https://cgap.nci.nih.gov/Pathways/BioCarta/h_a...,P,252,2,0,.2761904761904761904761904761904761904762,2.143572e-07,2,0.000024
2,WAG004117,BCDX2 complex stabilizes RAD51 filament,Reactome_2021,22,Homo sapiens,BCDX2 complex stabilizes RAD51 filament,https://reactome.org/PathwayBrowser/#/R-HSA-56...,P,13163,2,3370.35333598828,.2727272727272727272727272727272727272727,2.357930e-07,3,0.000026
3,WAG004154,"D-loop formation mediated by PALB2, BRCA2 and ...",Reactome_2021,24,Homo sapiens,"D-loop formation mediated by PALB2, BRCA2 and ...",https://reactome.org/PathwayBrowser/#/R-HSA-56...,P,13163,2,3436.07251714788,.2666666666666666666666666666666666666667,2.817267e-07,4,0.000031
4,WAG004162,"Defective D-loop formation mediated by PALB2, ...",Reactome_2021,24,Homo sapiens,"Defective D-loop formation mediated by PALB2, ...",https://reactome.org/PathwayBrowser/#/R-HSA-97...,P,13163,2,3436.07251714788,.2666666666666666666666666666666666666667,2.817267e-07,4,0.000030
5,WAG004163,"Defective D-loop formation mediated by PALB2, ...",Reactome_2021,24,Homo sapiens,"Defective D-loop formation mediated by PALB2, ...",https://reactome.org/PathwayBrowser/#/R-HSA-97...,P,13163,2,3436.07251714788,.2666666666666666666666666666666666666667,2.817267e-07,4,0.000030
6,WAG004164,"Defective D-loop formation mediated by PALB2, ...",Reactome_2021,24,Homo sapiens,"Defective D-loop formation mediated by PALB2, ...",https://reactome.org/PathwayBrowser/#/R-HSA-97...,P,13163,2,3436.07251714788,.2666666666666666666666666666666666666667,2.817267e-07,4,0.000030
7,WAG004118,CX3 complex binds D-loop structures,Reactome_2021,25,Homo sapiens,CX3 complex binds D-loop structures,https://reactome.org/PathwayBrowser/#/R-HSA-56...,P,13163,2,3687.28839591364,.264,3.062246e-07,8,0.000032
8,WAG004123,Resolution of D-loops cleaved by MUS81:EME1 or...,Reactome_2021,25,Homo sapiens,Resolution of D-loops cleaved by MUS81:EME1 or...,https://reactome.org/PathwayBrowser/#/R-HSA-56...,P,13163,2,3687.28839591364,.264,3.062246e-07,8,0.000032
9,WAG004124,Resolution of Holliday junctions cleaved by GE...,Reactome_2021,25,Homo sapiens,Resolution of Holliday junctions cleaved by GE...,https://reactome.org/PathwayBrowser/#/R-HSA-56...,P,13163,2,3687.28839591364,.264,3.062246e-07,8,0.000032


In [ ]:
# pathMember is a function connected to PAGER api to retrieve the membership of PAGs using a list of PAG IDs
# example:
PAG_member = PAGER.pathMember(list(PAG_result.GS_ID))
PAG_member

,GS_ID,GENE_SYM
0,WAG000212,FANCA
1,WAG000212,FANCC
2,WAG000212,FANCD2
3,WAG000212,FANCE
4,WAG000212,FANCF
...,...,...
3233,WIG001826,PLCH2
3234,WIG001826,CUL7
3235,WIG001826,RBX1
3236,WIG001826,CDC42


In [ ]:
# pathInt is a function connected to PAGER api to retrieve the m-type relationships of PAGs using a list of PAG IDs
# example:
PAGint = PAGER.pathInt(list(PAG_result.GS_ID))
PAGint

,GS_A_ID,GS_A_SIZE,GS_B_ID,GS_B_SIZE,OLAP,SIMILARITY,PVALUE
0,WAG004141,34,WAG004110,8,8,.3882352941176471,56.46586932342304
1,WAG004141,34,WAG003110,120,18,.21176470588235294,82.21351072887433
2,WAG004141,34,WAG004122,28,18,.4558441558441558,114.21936937441592
3,WAG004121,26,WAG004146,32,25,.7983682983682984,185.5314011339766
4,WAG004121,26,WAG004118,25,25,.9692307692307692,200.56039586011016
...,...,...,...,...,...,...,...
1038,WAG004110,8,WAG002125,157,5,.15,23.136492336027914
1039,WAG004110,8,WIG001825,157,5,.15,23.136492336027914
1040,WAG004110,8,WAG004120,16,8,.6000000000000001,63.71773790125735
1041,WAG004110,8,WAG004152,8,8,1,73.18039220187346


In [ ]:
# pathReg is a function connected to PAGER api to retrieve the r-type relationships of PAGs using a list of PAG IDs
# example:
PAGreg = PAGER.pathReg(list(PAG_result.GS_ID))
PAGreg

,GS_A_ID,GS_B_ID,GS_N,GS_A_OUT,GS_B_IN,AB,ABLOGCDF
0,WIG001819,WAG003221,2708,1976,1,1,1.30818512007307
1,WAG004141,WAG013979,1361,1213,1,1,2.2187627288787857
2,WAG002505,WAG013979,1786,1422,1,1,1.5905798938058977
3,WAG002125,WAG013979,8475,6335,1,1,1.376314824169259
4,WAG002505,WAG013979,1786,1422,1,1,1.5905798938058977
5,WIG001819,WAG013979,3715,2877,1,1,1.489115856653614
6,WAG002700,WAG013979,3859,3235,1,1,1.822012993181766
7,WAG004141,WAG013979,1361,1213,1,1,2.218762728878777
8,WIG001822,WAG013979,2425,1820,1,1,1.3883583453435069
9,WAG002700,WAG013979,3859,3235,1,1,1.822012993181766


In [ ]:
# pagRankedGene is a function connected to PAGER api to retrieve RP-ranked genes with RP-score of the given PAG_IDs
# example:
PAGRankedGenes = PAGER.pagRankedGene(list(PAG_result.GS_ID)[0])
PAGRankedGenes

,GS_ID,GENE_SYM,RP_SCORE,ENTREZ_ID,DESCRIPTION
0,WAG002693,MIR3065,None,100422915,microRNA 3065
1,WAG002693,TP53,2.94247626720678,7157,tumor protein p53
2,WAG002693,BRCA1,1.99979907273332,672,BRCA1 DNA repair associated
3,WAG002693,BRCA2,1.98761830158018,675,BRCA2 DNA repair associated
4,WAG002693,MMP2,.954919869629295,4313,matrix metallopeptidase 2


In [ ]:
# pagGeneInt is a function connected to PAGER api to retrieve gene interaction network
# example:
# pagGeneInts = PAGER.pagGeneInt(list(PAG_result.GS_ID)[0])
# pagGeneInts

all_pagGeneInts = pd.concat([PAGER.pagGeneInt(gs_id) for gs_id in list(PAG_result.GS_ID)], ignore_index=True)
all_pagGeneInts
pag_graph = WeightedGraph()
for node1, node2, weight in all_pagGeneInts.values.tolist():
  pag_graph.add_edge(node1, node2, weight)

print(pag_graph)

,SYM_A,SYM_B,H_SCORE
0,BRCA1,BRCA2,.9999
1,BRCA1,TP53,1
2,BRCA2,TP53,.9939
3,MMP2,TP53,.9772


In [ ]:
# pagGeneReg is a function connected to PAGER api to retrieve gene regulatory network
# example:
pagGeneRegs = PAGER.pagGeneReg(list(PAG_result.GS_ID)[0])
pagGeneRegs

,A_ID,B_ID,SCR,MACHANISM
0,TP53,BRCA1,TRED,None
1,TP53,MMP2,TRED,None
2,TP53,BRCA2,STRING v9.05,Inhibition
3,TP53,BRCA2,TRED,None
4,TP53,TP53,STRING v9.05,Inhibition
5,TP53,TP53,STRING v9.05,Activation
6,TP53,TP53,TRED,None
7,BRCA1,TP53,TRED,None
8,TP53,MMP2,Spike,Transcription Regulation
9,TP53,MMP2,TRANSFAC,None


In [ ]:
# path_NGSEA is a function connected to PAGER api to generate the network-based GSEA result
# example:
gene_df = pd.DataFrame({'symbol':["BRCA1","BRCA2","RAD51","PGR"], 'log2FoldChange':['0.6','0.3','0.1','0.1']})
pagInNetworkWGSEA = PAGER.path_NGSEA(gene_df,PAG_member)
pagInNetworkWGSEA

,Term,es,nes,pval,fdr,geneset_size,matched_size,genes,ledge_genes
0,WAG002840,nan,nan,0.0,0.097570,155,4,BRCA1;BRCA2;RAD51;PGR,BRCA1;BRCA2;RAD51;PGR
1,WAG014173,1.0,2.318392581143741,0.5077519379844961,0.424741,6,1,BRCA1,BRCA1
2,WAG004152,1.0,2.332814930015552,0.4826254826254826,0.426504,8,1,BRCA1,BRCA1
3,WAG014925,1.0,2.332814930015552,0.531496062992126,0.426504,2,1,BRCA1,BRCA1
4,WIG001826,1.0,2.3364485981308416,0.5078125,0.431136,448,1,BRCA1,BRCA1
5,WAG002161,1.0,2.269977663637762,0.3179916317991632,0.433949,83,2,BRCA1;BRCA2,BRCA1;BRCA2
6,WAG014929,1.0,2.34375,0.490272373540856,0.433949,3,1,BRCA1,BRCA1
7,WAG004120,1.0,2.3961661341853038,0.5140562248995983,0.438121,16,1,BRCA1,BRCA1
8,WAG014927,1.0,2.4077046548956664,0.492,0.442487,2,1,BRCA1,BRCA1
9,WAG014931,1.0,2.272727272727273,0.5095057034220533,0.444734,3,1,BRCA1,BRCA1
